In [1]:
from typing import List
import os

import pandas as pd
import numpy as np
from cobra.io import read_sbml_model
from cobra.io.sbml import Model
from mewpy.simulation import Environment, Simulator, get_simulator

In [2]:
path = '../models/ec/'
model_list: List = os.listdir(path)
m_list: List = []

for model in model_list:
    model = path + model
    m_list.append(model)

print('Models:', m_list)
l_va = np.linspace(0.1, 1.0, num=10)
constraint_list: List = []

for i in range(len(l_va)):
    constraint_list.append({'prot_pool_exchange': (0, round(l_va[i], 1))})
print('Conditions:', constraint_list)

Models: ['../models/ec/ec_Streptococcus_parasanguinis_ATCC_15912.xml', '../models/ec/ec_Coprococcus_comes_ATCC_27758.xml', '../models/ec/ec_Bacteroides_thetaiotaomicron_VPI_5482.xml', '../models/ec/ec_Bacteroides_uniformis_ATCC_8492.xml', '../models/ec/ec_Fusobacterium_nucleatum_subsp_nucleatum_ATCC_25586.xml', '../models/ec/ec_Escherichia_coli_ED1a.xml', '../models/ec/ec_Roseburia_intestinalis_L1_82.xml', '../models/ec/ec_Streptococcus_salivarius_DSM_20560.xml']
Conditions: [{'prot_pool_exchange': (0, 0.1)}, {'prot_pool_exchange': (0, 0.2)}, {'prot_pool_exchange': (0, 0.3)}, {'prot_pool_exchange': (0, 0.4)}, {'prot_pool_exchange': (0, 0.5)}, {'prot_pool_exchange': (0, 0.6)}, {'prot_pool_exchange': (0, 0.7)}, {'prot_pool_exchange': (0, 0.8)}, {'prot_pool_exchange': (0, 0.9)}, {'prot_pool_exchange': (0, 1.0)}]


In [4]:
for m_path in m_list:
    sol_list = []
    model = read_sbml_model(m_path)
    model_id = str(model.id.strip('M_'))
    sim: Simulator = get_simulator(model)
    env = Environment.complete(sim, max_uptake=1000.0, inplace=False)
    sim.set_environmental_conditions(env)
    solution = sim.simulate(constraints=cons)
    sol_list.append([cons, {f'{model_id}': solution.objective_value}])

[[{'prot_pool_exchange': (0, 0.1)}, {'Streptococcus_parasanguinis_ATCC_15912': 0.00014375934065246381}], [{'prot_pool_exchange': (0, 0.2)}, {'Streptococcus_parasanguinis_ATCC_15912': 0.00028751868130492763}], [{'prot_pool_exchange': (0, 0.3)}, {'Streptococcus_parasanguinis_ATCC_15912': 0.0004312780219573914}], [{'prot_pool_exchange': (0, 0.4)}, {'Streptococcus_parasanguinis_ATCC_15912': 0.0005750373626098553}], [{'prot_pool_exchange': (0, 0.5)}, {'Streptococcus_parasanguinis_ATCC_15912': 0.0007187967032623191}], [{'prot_pool_exchange': (0, 0.6)}, {'Streptococcus_parasanguinis_ATCC_15912': 0.0008625560439147828}], [{'prot_pool_exchange': (0, 0.7)}, {'Streptococcus_parasanguinis_ATCC_15912': 0.0010063153845672466}], [{'prot_pool_exchange': (0, 0.8)}, {'Streptococcus_parasanguinis_ATCC_15912': 0.0011500747252197105}], [{'prot_pool_exchange': (0, 0.9)}, {'Streptococcus_parasanguinis_ATCC_15912': 0.0012938340658721745}], [{'prot_pool_exchange': (0, 1.0)}, {'Streptococcus_parasanguinis_ATCC_

OptimizationError: Solver status is 'unbounded'.